In [ ]:
import numpy as np
import time
from Helper import LearningCurvePlot, smooth
from Assignment2_batch_updates import *

def average_over_repetitions(policy, epsilon, smoothing_window, plot, decay_epsilon, n_repetitions, n_runs, learning_rate, gamma, batch_size):

    reward_results = np.empty([n_repetitions,n_runs]) # Result array
    now = time.time()
    
    for rep in range(n_repetitions): # Loop over repetitions
        rewards = cartpole(n_runs, learning_rate, gamma, policy, epsilon, experience_replay, batch_size, decay_epsilon)
        reward_results[rep] = rewards
        
    print('Running one setting takes {} minutes'.format((time.time()-now)/60))    
    learning_curve = np.mean(reward_results,axis=0) # average over repetitions
    learning_curve = smooth(learning_curve,smoothing_window) # additional smoothing
    return learning_curve  

def experiment():
    ####### Settings
    # Experiment    
    n_repetitions = 5
    smoothing_window = 1001
    n_runs = 65
    gamma = 0.995

    # Exploration
    policy = 'egreedy' # 'egreedy' or other choices
    epsilon_min = 0.01
    decay_rate = 0.95
    batch_size = 64
    decay_epsilon = False
    
    # Learning rate
    learning_rate = 0.01
    
    # Plotting parameters
    plot = True
    
    # Nice labels for plotting
    policy_labels = {'egreedy': '$\epsilon$-greedy policy',
                  'softmax': 'Softmax policy'}                  #Something here might need to be changed

    
    ####### Experiments
    
    good_average_reward = 350 # We set this as a benchmark of good average reward reached by the algorithm
    
    Plot = LearningCurvePlot(title = 'Cartpole experiment with $\epsilon$-greedy exploration')    
    policy = 'egreedy'
    epsilons = [0.01, 1 ,0.2]
    for epsilon in epsilons:
        if epsilon == 1:
            decay_epsilon = True
        learning_curve = average_over_repetitions(policy, epsilon, smoothing_window, plot, decay_epsilon, 
                                                  n_repetitions, n_runs, learning_rate, gamma, batch_size)
        Plot.add_curve(learning_curve,label=r'$\epsilon$-greedy, $\epsilon $ = {}'.format(epsilon))
        decay_epsilon = False
    #policy = 'softmax'
    #temps = [0.01,0.1,1.0]
    #for temp in temps:
     #   learning_curve = average_over_repetitions(backup, n_repetitions, n_timesteps, max_episode_length, learning_rate, 
      #                                        gamma, policy, epsilon, temp, smoothing_window, plot, n)
       # Plot.add_curve(learning_curve,label=r'softmax, $ \tau $ = {}'.format(temp))
    Plot.add_hline(optimal_average_reward_per_timestep)
    Plot.save('cartpole_test.png')

if __name__ == '__main__':
    experiment()


pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
Run:1, Steps:9, Epsilon:0.01
Run:2, Steps:11, Epsilon:0.01
Run:3, Steps:10, Epsilon:0.01
Run:4, Steps:8, Epsilon:0.01
Run:5, Steps:10, Epsilon:0.01
Run:6, Steps:9, Epsilon:0.01
Run:7, Steps:13, Epsilon:0.01
Run:8, Steps:10, Epsilon:0.01
Run:9, Steps:9, Epsilon:0.01
Run:10, Steps:10, Epsilon:0.01
Run:11, Steps:10, Epsilon:0.01
Run:12, Steps:9, Epsilon:0.01
Run:13, Steps:10, Epsilon:0.01
Run:14, Steps:11, Epsilon:0.01
Run:15, Steps:10, Epsilon:0.01
Run:16, Steps:14, Epsilon:0.01
Run:17, Steps:97, Epsilon:0.01
Run:18, Steps:23, Epsilon:0.01
Run:19, Steps:28, Epsilon:0.01
Run:20, Steps:33, Epsilon:0.01
Run:21, Steps:27, Epsilon:0.01
Run:22, Steps:28, Epsilon:0.01
Run:23, Steps:69, Epsilon:0.01
Run:24, Steps:118, Epsilon:0.01
Run:25, Steps:56, Epsilon:0.01
Run:26, Steps:69, Epsilon:0.01
Run:27, Steps:53, Epsilon:0.01
Run:28, Steps:68, Epsilon:0.01
Run:29, Steps:67, Epsilon:0.01
Ru